In [110]:
import pandas as pd
import numpy as np
import plotly.express as px
from statsmodels .tsa.api import ExponentialSmoothing, SimpleExpSmoothing
import time


In [111]:
#Importing necessary data which has been obtained via UN source. Pulled all import and export data available for selected country.

china = pd.read_csv('https://raw.githubusercontent.com/jacobaschoff/midterm/main/New%20China%20Data%20(Commodity%20Coded).csv') #Reading in CSV file.
china = pd.DataFrame(china) #Dataframe to enable easier Pandas library usage.
china

,Country or Area,Year,Comm. Code,Commodity,Flow,Trade (USD),Weight (kg),Quantity Name,Quantity
0,China,2022,290110,Acyclic hydrocarbons; saturated,Export,91818482,5.609180e+07,Weight in kilograms,5.609180e+07
1,China,2022,290110,Acyclic hydrocarbons; saturated,Import,2094540206,3.256437e+09,Weight in kilograms,3.256437e+09
2,China,2022,290121,"Acyclic hydrocarbons; unsaturated, ethylene",Import,2165546765,2.067549e+09,Weight in kilograms,2.067549e+09
3,China,2022,290121,"Acyclic hydrocarbons; unsaturated, ethylene",Export,162221671,1.540843e+08,Weight in kilograms,1.540843e+08
4,China,2022,290122,"Acyclic hydrocarbons; unsaturated, propene (pr...",Import,2379997192,2.337133e+09,Weight in kilograms,2.337133e+09
...,...,...,...,...,...,...,...,...,...
18856,China,1992,294150,Antibiotics; erythromycin and its derivatives;...,Import,1129587,2.831800e+04,Weight in kilograms,2.831800e+04
18857,China,1992,294190,Antibiotics; n.e.c. in heading no. 2941,Export,102557482,4.088511e+06,Weight in kilograms,4.088511e+06
18858,China,1992,294190,Antibiotics; n.e.c. in heading no. 2941,Import,103517274,1.002747e+06,Weight in kilograms,1.002747e+06
18859,China,1992,294200,Organic compounds; n.e.c. in chapter 29,Export,4441397,1.200874e+06,Weight in kilograms,1.200874e+06


In [112]:
fentanylData = china[china['Comm. Code'] == 293333]
exportData = fentanylData[fentanylData['Flow'] == 'Export']
importData = fentanylData[fentanylData['Flow'] == 'Import']

In [113]:
figChinaExValue = px.line(exportData, x='Year', y='Trade (USD)')
figChinaExValue

In [114]:
df = exportData.groupby(['Year'])['Weight (kg)'].sum()

In [115]:
data = df.values
data

array([1.63000e+02, 1.81500e+03, 3.11400e+03, 1.35350e+04, 1.42600e+03,
       6.99000e+02, 1.24800e+03, 3.10000e+02, 2.04000e+02, 0.00000e+00,
       3.00000e+00, 5.08000e+02, 4.00000e+00, 2.51024e+02, 0.00000e+00,
       2.01100e+00, 3.28900e+00, 6.85700e+00, 1.68490e+01, 1.81040e+01,
       1.39660e+01])

In [116]:
alpha020 = SimpleExpSmoothing(data).fit(
                                        smoothing_level=0.2,
                                        optimized=False)

alpha050 = SimpleExpSmoothing(data).fit(
                                        smoothing_level=0.5,
                                        optimized=False)

alpha080 = SimpleExpSmoothing(data).fit(
                                        smoothing_level=0.8,
                                        optimized=False)

level2 = alpha020.forecast(1)
level5 = alpha050.forecast(1)
level8 = alpha080.forecast(1)

print(level2, level5, level8)

[142.73620772] [15.82673659] [14.66000561]


In [117]:
levels = pd.DataFrame([data, 
[float(level2) for i in range(21)], 
[float(level5) for i in range(21)], 
[float(level8) for i in range(21)]]).T

levels.columns = ['fentanyl_seizures', 'alpha020', 'alpha050', 'alpha080']

levels.reset_index(drop=True, inplace=True)

# Plot using Plotly
fig = px.line(levels, y=['fentanyl_seizures', 'alpha020', 'alpha050', 'alpha080'])

# Show the plot
fig.show()


/var/folders/9g/yqwm3f7s7m9dpll0wmghzmkw0000gn/T/ipykernel_35417/2935812083.py:2: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)

/var/folders/9g/yqwm3f7s7m9dpll0wmghzmkw0000gn/T/ipykernel_35417/2935812083.py:3: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)

/var/folders/9g/yqwm3f7s7m9dpll0wmghzmkw0000gn/T/ipykernel_35417/2935812083.py:4: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



In [118]:
trend = ExponentialSmoothing(data, trend='add', damped_trend=True).fit()
forecast = trend.forecast(21)
values = np.concatenate([data, forecast])
yesnoforecast = [0] * 21 + [1] * 21
trends = pd.DataFrame({'exports': values, 'forecasted': yesnoforecast})
trends

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.



,exports,forecasted
0,163.000000,0
1,1815.000000,0
2,3114.000000,0
3,13535.000000,0
4,1426.000000,0
5,699.000000,0
6,1248.000000,0
7,310.000000,0
8,204.000000,0
9,0.000000,0


In [119]:
px.line(trends, y='exports', color='forecasted')

In [120]:
# https://www.cbp.gov/newsroom/local-media-release/philadelphia-cbp-seizes-nearly-17-million-fentanyl-shipped-china
# article suggests $34k per kg
lb_price = 34000/2.20462
twentyfour = forecast[1]/lb_price

In [121]:
1700000/15000

113.33333333333333